In [ ]:

# Fruit Freshness Classification using Fruits-360 Dataset in Google Colab

# Step 1: Download and unzip dataset
!wget -q --show-progress https://github.com/Horea94/Fruit-Images-Dataset/archive/refs/heads/master.zip -O fruits360.zip
!unzip -q fruits360.zip

# Step 2: Check folder structure
import os
DATA_DIR = "Fruit-Images-Dataset-master/Training"
print("Classes:", os.listdir(DATA_DIR))

# Step 3: Load images and labels function (only using 2 classes for demo)
import cv2
import numpy as np

IMG_SIZE = 100
def load_data(classes_to_use):
    X = []
    y = []
    classes = classes_to_use
    for label in classes:
        path = os.path.join(DATA_DIR, label)
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                X.append(img)
                y.append(classes.index(label))
    return np.array(X), np.array(y)

# For simplicity, just use these two classes
selected_classes = ['Apple Crimson Snow', 'Apple Golden 1']

X, y = load_data(selected_classes)
X = X / 255.0

print("Loaded images shape:", X.shape)
print("Labels shape:", y.shape)

# Step 4: Train/test split & model training
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(selected_classes), activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

loss, acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {acc:.4f}")
